# AiDM Assignment 2 - LSH for the Netflix data
## Naive method - Time esitimator

### Read data
First of all, we load in the data of approximately 103,703 users and the movies they watched among 17,770 movies in total. The first column gives the user IDs and the second column gives the movies IDs.

In [1]:
import numpy as np
import time
time0 = time.time() # record the starting time of loading part.
data_origin = np.load('../user_movie.npy')
loading_time = time.time() - time0     # calculate the time of this part.
print('loading time = ',loading_time,'sec')

loading time =  0.22000908851623535 sec


As wee can see here, our loading process took about 0.21 second.

### List and calculate the similarities
Theoretically, This naive algorithm consists of two parts. Firstly, we would like to make a movie set for every single user and then put them in to a list. Secondly, for each user's movie set, we compare it with another user's.

In [ ]:
def generate_list(data,n):
    '''
    The function is to generate a list that contains sets for users. Each set is the watched movies for one user.
    data :
    n : persent of dataset will be considered
    '''
    data_n_persent = data[0:int(len(data_origin)*n),:] # Drag out n persent from the entire dataset.
    user_ID = np.unique(data_n_persent[:,0])      # Make a user ID array, every single ID will only show once.
    user_movie_list = []                     # Prepare an empty watched movie list
    for i in range(len(user_ID)):
        # For every user, we go through the data and record the movies that he/she watched.
        marker = data[:,0] == user_ID[i]
        user_movie_list.append(set(data[marker][:,1]))   # Convert it into set in order to compare with others'
    return user_movie_list  # will return the whole watched movies set for every user in only one list.


def simi(set_list):
    '''
    This function is to calculate the similarities between users.
    Similarity = intersenction / union
    set_list : 
    '''
    similarities = np.zeros(len(set_list)) # Prepare an np array for saving the value of similarities.
    for i in range(len(set_list)):
        intersection = set_list[0]&set_list[i]  # Use logical and to achieve the intersection
        similarities[i] = len(intersection)/(len(set_list[0])+len(set_list[i])-len(intersection)) # Calculate sim
    return similarities # will return a array contains similarities between the first user and any others.

### Record running time
We did a test on 1 persent of the entire dataset and recorded the running time of listing and calculating the similarities separately. 

In [63]:
time1 = time.time()    # Record the starting point of listing process.
set_1persent = generate_list(data_origin,0.01)
list_time = time.time() - time1       #  Calculate the time of this part.
print('listing time = ',list_time,'sec')

138.52079319953918 seconds


The time it took to generate a list, which contains movie set of users, from 1 persent of the dataset is about 138.52 seconds. Note that we chop out the first 1 persent of data based on rows rather than users which means the information of the last user in our test might not be completed but this issue will not affect our result of estimated time. Hence, we achieved a list consisting 1040 users' movie set.

In [64]:
time2 = time.time()       # Record the staring time of similarities calculation process.
simi_1persent = simi(set_1persent)
simi_time = time.time() - time2   # calculate the time of this part
print('similarities calculating time = ',simi_time,'sec')

0.0317378044128418 seconds


According to our timer, the calculating process took about 0.032 to run. The number of calculations is as the same as the number of users as we compared the first user with others ( to simplify the code, we campared the first user with himself), which is 1040.

### Esitimate the total  time
Then we estimated the total time by considering 3 parts of the entire algorithm. 

First of all, the time cost by loading data process is fixed, which is aroud 0.25 sec. 

Second, the listing part is a linear opertation. Our test was making a list on 1040 users from 103,703 users. Hence, the total estimated time of second part is supposed to be about 100 times of it for our test, which is 103,703/1040 * 138.52 sec.

Third, the similarities part. The time of this part is proportional to the number of calculations. In our case, we calculated 1040 similarities. The total dataset needs 103,703 * 103,702 / 2 times calculations. Therefore, we multipiled our test time by (103,703 * 103,702 /2) / 1040.

In [80]:
num_users = len(set(data_origin[:,0]))   # The  total number of users
num_users_1p = len(set_1persent)         # The numbers of users in our test
opertaions_toal = num_users * (num_users - 1) / 2.   # Total calculations 
opertaions_1p = num_users_1p                         # Calculations of our test
op_factor = opertaions_toal / opertaions_1p
total_time = loading_time + list_time * 100. + simi_time * op_factor    # Add them up
print('Total time = ', total_time / 60. / 60. / 24., ' d')

Total time =  2.0595609208055614  d


In [5]:
max(data_origin[:,1])*5

88845

Our final result of total estimated time is about 2 days.